In [1]:
import gmsh
import sys
import os
from os.path import exists
import subprocess
import numpy as np
import time
import ROOT
from os import listdir
from os.path import isfile, join
import os.path
from os import path
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import argparse
from ROOT import TFile, TTree
import matplotlib.pyplot as plt
from scipy.interpolate import splrep, splev,splprep
from scipy.interpolate import BSpline
import math

Welcome to JupyROOT 6.26/10


In [2]:
import math
from typing import List
import contextlib
import io
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import ROOT
import numpy as np
from matplotlib.lines import Line2D


def checkRootData(fileName, treeName, branchName):
    try:
        file = ROOT.TFile.Open(fileName)

        # Check if file is open
        if not file or file.IsZombie():
            print("File is Zombie")
            return False

        # Check if tree exists
        if not file.Get(treeName):
            print("Tree does not exist")
            return False

        tree = file.Get(treeName)

        # Check if branch exists
        if not tree.GetBranch(branchName):
            print("Branch name does not exist")
            return False

        branch = tree.GetBranch(branchName)

        # Check if branch is readable
        value = np.zeros(1, dtype=np.float64)
        branch.SetAddress(value)

        if branch.GetEntry(0) < 0:
            print("Branch is not readable")
            return False

        file.Close()

        return True

    except:
        print("An error occurred while checking root data")
        return False
    
def readRootBranch(fileName, treeName, branchName):
    print("fileName:", fileName)

    data = []

    if checkRootData(fileName, treeName, branchName):
        print(checkRootData(fileName, treeName, branchName))
        file = ROOT.TFile.Open(fileName)
        tree = file.Get(treeName)
        branch = tree.GetBranch(branchName)

        value = array('d', [0.0])
        branch.SetAddress(value)

        for i in range(tree.GetEntries()):
            branch.GetEntry(i)
            data.append(value[0])

        file.Close()
    else:
        print("Something is wrong with the file!!!")
        print("Branch:", branchName)

    return data

import os
from os import path
from typing import List
from array import array

def readRootDataFromFolder(folder, treeName, branchName):
    data = []

    # Open folder
    if os.path.isdir(folder):
        # Read all files in folder
        for file_name in os.listdir(folder):
            file_path = os.path.join(folder, file_name)
            # Skip directories
            if os.path.isdir(file_path):
                continue

            # Read data from filea
            file_data = readRootBranch(file_path, treeName, branchName)
            data.extend(file_data)

    return data
import matplotlib.pyplot as plt
import math
import numpy as np
from scipy.optimize import curve_fit
import contextlib
import io

In [3]:
import matplotlib as mpl

def setup_pgf():
    pgf_with_latex = {
        "pgf.texsystem": "pdflatex",
        "text.usetex": True,
        "font.family": "serif",
        "font.serif": [],
        "font.sans-serif": [],
        "font.monospace": [],
        "axes.labelsize": 10,
        "font.size": 10,
        "legend.fontsize": 6,
        "xtick.labelsize": 10,
        "ytick.labelsize": 10,
        "pgf.preamble": "\n".join([
            r"\usepackage[utf8]{inputenc}",
            r"\usepackage[T1]{fontenc}",
            r"\usepackage{amsmath}",
            r"\usepackage[detect-all]{siunitx}",
            r"\newcommand{\xbar}{\bar{x}}",
        ])
    }
    mpl.use("pgf")
    mpl.rcParams.update(pgf_with_latex)
 # Set the figure size

    # Set the figure size based on LaTeX geometry settings
    geometry_textwidth = 384  # pt
    pt_to_inch = 1.0 / 72.27  # Conversion factor from points to inches
    plot_width = geometry_textwidth * pt_to_inch *0.5
    
    mpl.rcParams['figure.figsize'] = (plot_width, plot_width*0.9)  # Adjust the height as needed (6 inches in this example)
def restore_defaults():
    mpl.rcParams.update(mpl.rcParamsDefault)
setup_pgf()

# Comparison of the overall distributions with boxplots of multiple optimization folders and objectives

In [6]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import contextlib
import io

# Automatic reading of given folders for the results (local computer definition)
# returns a list of lists with all the data of each folder for the given branch
def get_data_lists(branch):
    data = []

    with contextlib.redirect_stdout(io.StringIO()):
        Def_folder="../TierIIData/2023_07/Default_FL_Muon_ESRBP"
        Def = readRootDataFromFolder(Def_folder, "EndOfEvent", branch)
        data.append(Def)

        LO_Avg_folder="../NSGAIIData/2307/Solutions/Sol_LOAvg"
        LO_Avg = readRootDataFromFolder(LO_Avg_folder, "EndOfEvent", branch)
        data.append(LO_Avg)

        LO_P50_folder="../NSGAIIData/2307/Solutions/Sol_LOP50"
        LO_P50 = readRootDataFromFolder(LO_P50_folder, "EndOfEvent", branch)
        data.append(LO_P50)

        LC_Avg_folder="../NSGAIIData/2307/Solutions/Sol_LCAvg"
        LC_Avg = readRootDataFromFolder(LC_Avg_folder, "EndOfEvent", branch)
        data.append(LC_Avg)

        LC_P50_folder="../NSGAIIData/2307/Solutions/Sol_LCP50"
        LC_P50 = readRootDataFromFolder(LC_P50_folder, "EndOfEvent", branch)
        data.append(LC_P50)

        LST_Avg_folder="../NSGAIIData/2307/Solutions/Sol_LStAvg"
        LST_Avg = readRootDataFromFolder(LST_Avg_folder, "EndOfEvent", branch)
        data.append(LST_Avg)

        LST_P50_folder="../NSGAIIData/2307/Solutions/Sol_LStP50"
        LST_P50 = readRootDataFromFolder(LST_P50_folder, "EndOfEvent", branch)
        data.append(LST_P50)

    return data

In [7]:
# Creates a boxplot at the locations given in x for each list in the list data
def create_boxplots(ax, fig, positions, data, labels, widths, linewidths,showfliers=True):
    assert len(positions) == len(data), "Input lists must have the same length"

    for pos, d in zip(positions, data):
        bp = ax.boxplot(d, positions=[pos], widths=widths, showfliers=showfliers,
                        patch_artist=False,
                        whiskerprops=dict(color=color, linewidth=linewidths),
                        capprops=dict(color=color, linewidth=linewidths),
                        medianprops=dict(color=color, linewidth=linewidths),
                        flierprops=dict(marker='o', markersize=1, markerfacecolor='black'),
                        whis=1.5,
                        boxprops=dict(color=color, linewidth=0.5))

        # Calculate the average of the data set
        avg = np.mean(d)
        if pos==0:
            ax.scatter(pos, avg, color='red', label='Mean', s=6)
        else:
            ax.scatter(pos, avg, color='red', s=6)
            print(pos,"Avgs:",avg)
            print(pos,"P_50:",np.median(d))
        # Add a red point scatter for the average
        

    # Add a grid
    ax.grid(True)
    # Set x-axis ticks as letters
    ax.set_xticklabels(labels, rotation=45)
    # Set y-axis label

# Default x axis given the folders used
labels = ['Def.','$\\overline{L_O}$', '$\\widetilde{L_O}$', '$\\overline{L_C}$', '$\\widetilde{L_C}$', '$\overline{L_{SP}}$', '$\\widetilde{L_{SP}}$']

In [4]:
# Example usage:
branch = "fLOapprox"
data = get_data_lists(branch)

Error in <TFile::Init>: file ../NSGAIIData/2307/Solutions/Sol_LOP50/LO_G200_P50Opt_Vol200_230805_1core_30.root has an incorrect header length (264) or incorrect end of file length (246)
Error in <TFile::Init>: file ../NSGAIIData/2307/Solutions/Sol_LStAvg/LSt_G46_AvgOpt_Vol513_230805_1core_30.root has an incorrect header length (264) or incorrect end of file length (246)


In [8]:
# Example usage:
branch = "fLOapprox"
data = get_data_lists(branch)

geometry_textwidth = 384  # pt
pt_to_inch = 1.0 / 72.27  # Conversion factor from points to inches
plot_width = geometry_textwidth * pt_to_inch *0.5
fig, ax = plt.subplots(figsize=(plot_width, plot_width*0.9))
color='blue'
widths=[0.8]

# Create the boxplot
positions = [0, 1, 2, 3, 4, 5, 6]
create_boxplots(ax,fig,positions, data, labels, widths,1)
ax.set_ylabel(r"$L_O$ $\left(\si{\gamma_d/\mega\electronvolt}\right)$")  # Set x-axis label with units
ax.scatter([1,2], [3062,3497],marker='x', color='blue', label='NSGAII', s=20)

ax.legend()
#ax.scatter([1,2], [3062,3497],marker='*', color='darkblue', label='Mean', s=20)
plt.savefig("Figures/2307/NSGAII/RstLOComp.pdf", bbox_inches='tight')
plt.show()

1 Avgs: 3066.107277031986
1 P_50: 3062.544324084013
2 Avgs: 3300.0166884912983
2 P_50: 3477.9961421219377
3 Avgs: 2703.0594548145727
3 P_50: 2721.3966094443717
4 Avgs: 2622.349914418523
4 P_50: 2605.1271189226
5 Avgs: 2789.6155499223823
5 P_50: 2791.9335138131046
6 Avgs: 2498.9432553949027
6 P_50: 2527.920079944972


ValueError: 
\left(\si{\gamma_d/\mega\electronvolt}\right)
      ^
Expected '\\right', found '\'  (at char 6), (line:1, col:7)

Error in callback <function _draw_all_if_interactive at 0x7f3a9e42aa60> (for post_execute):


Error in <TFile::Init>: file ../NSGAIIData/2307/Solutions/Sol_LOP50/LO_G200_P50Opt_Vol200_230805_1core_30.root has an incorrect header length (264) or incorrect end of file length (246)
Error in <TFile::Init>: file ../NSGAIIData/2307/Solutions/Sol_LStAvg/LSt_G46_AvgOpt_Vol513_230805_1core_30.root has an incorrect header length (264) or incorrect end of file length (246)


ValueError: 
\left(\si{\gamma_d/\mega\electronvolt}\right)
      ^
Expected '\\right', found '\'  (at char 6), (line:1, col:7)

ValueError: 
\left(\si{\gamma_d/\mega\electronvolt}\right)
      ^
Expected '\\right', found '\'  (at char 6), (line:1, col:7)

<Figure size 265.67x239.103 with 1 Axes>

In [7]:
# Example usage:
branch = "fHits"
data = get_data_lists(branch)

geometry_textwidth = 384  # pt
pt_to_inch = 1.0 / 72.27  # Conversion factor from points to inches
plot_width = geometry_textwidth * pt_to_inch *0.5
fig, ax = plt.subplots(figsize=(plot_width, plot_width*0.9))
color='blue'
widths=[0.8]
# Create the boxplot
positions = [0, 1, 2, 3, 4, 5, 6]
ax.scatter([3,4], [16780,14605],marker='x', color='blue', label='NSGAII', s=20)

create_boxplots(ax,fig,positions, data, labels, widths,1,showfliers=False)
ax.set_ylabel(r"$L_C$ $\left(\si{\gamma_d}\right)$")  # Set x-axis label with units
ax.legend()

plt.savefig("Figures/2307/NSGAII/RstLCComp.pdf", bbox_inches='tight')
plt.show()

1 Avgs: 6472.772
1 P_50: 5373.5
2 Avgs: 7240.585
2 P_50: 5849.0
3 Avgs: 16264.692
3 P_50: 14883.5
4 Avgs: 14999.452
4 P_50: 15059.0
5 Avgs: 16538.74
5 P_50: 15023.5
6 Avgs: 14593.556363636364
6 P_50: 16515.5


ValueError: 
\left(\si{\gamma_d}\right)
      ^
Expected '\\right', found '\'  (at char 6), (line:1, col:7)

Error in callback <function _draw_all_if_interactive at 0x7fe478b4a310> (for post_execute):


Error in <TFile::Init>: file ../NSGAIIData/2307/Solutions/Sol_LOP50/LO_G200_P50Opt_Vol200_230805_1core_30.root has an incorrect header length (264) or incorrect end of file length (246)
Error in <TFile::Init>: file ../NSGAIIData/2307/Solutions/Sol_LStAvg/LSt_G46_AvgOpt_Vol513_230805_1core_30.root has an incorrect header length (264) or incorrect end of file length (246)


ValueError: 
\left(\si{\gamma_d}\right)
      ^
Expected '\\right', found '\'  (at char 6), (line:1, col:7)

ValueError: 
\left(\si{\gamma_d}\right)
      ^
Expected '\\right', found '\'  (at char 6), (line:1, col:7)

<Figure size 265.67x239.103 with 1 Axes>

In [8]:
# Example usage:
branch = "fLSt"
data = get_data_lists(branch)
with contextlib.redirect_stdout(io.StringIO()):
    Def_folder="../TierIIData/2023_07/Default_FL_Muon_ESRBP"
    Def = readRootDataFromFolder(Def_folder, "EndOfEvent", "fLOapprox")
DefLSP=[i*3 for i in Def]
data[0]=DefLSP

geometry_textwidth = 384  # pt
pt_to_inch = 1.0 / 72.27  # Conversion factor from points to inches
plot_width = geometry_textwidth * pt_to_inch *0.5
fig, ax = plt.subplots(figsize=(plot_width, plot_width*0.9))
color='blue'
widths=[0.8]
# Create the boxplot
positions = [0, 1, 2, 3, 4, 5, 6]
ax.scatter([5,6], [8535,10031],marker='x', color='blue', label='NSGAII', s=20)

create_boxplots(ax,fig,positions, data, labels,widths,1)
ax.set_ylabel(r"$L_{SP}$ $\left(\si{\gamma_d\milli\meter/\mega\electronvolt}\right)$")  # Set x-axis label with units
ax.legend()


plt.savefig("Figures/2307/NSGAII/RstLSTComp.pdf", bbox_inches='tight')
plt.show()

1 Avgs: 3465.992633583639
1 P_50: 3006.924976761119
2 Avgs: 3778.9910775710655
2 P_50: 3211.7280145658647
3 Avgs: 8186.24392910223
3 P_50: 6769.142962782365
4 Avgs: 7615.410986136892
4 P_50: 8977.13996853961
5 Avgs: 8450.049243401669
5 P_50: 8032.725274288281
6 Avgs: 7588.453315488992
6 P_50: 9961.694035810915


/tmp/ipykernel_43149/2277096877.py:26: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
Error in <TFile::Init>: file ../NSGAIIData/2307/Solutions/Sol_LOP50/LO_G200_P50Opt_Vol200_230805_1core_30.root has an incorrect header length (264) or incorrect end of file length (246)
Error in <TFile::Init>: file ../NSGAIIData/2307/Solutions/Sol_LStAvg/LSt_G46_AvgOpt_Vol513_230805_1core_30.root has an incorrect header length (264) or incorrect end of file length (246)


# Now we want the box plots alog the impact in Z comparing the original and Avg(LST)

In [9]:
def get_data_lists(branch):
    data = []
    dataDef = []

    with contextlib.redirect_stdout(io.StringIO()):
        Def_folder="../TierIIData/2023_07/Default_FL_Muon_ESRBP"
        Def = readRootDataFromFolder(Def_folder, "EndOfEvent", branch)
        dataDef.append(Def)
        Def = readRootDataFromFolder(Def_folder, "EndOfEvent", 'fPrimaryZ')
        dataDef.append(Def)
        Opt_folder="../NSGAIIData/2307/Solutions/Sol_LStAvg"
        Opt = readRootDataFromFolder(Opt_folder, "EndOfEvent", branch)
        data.append(Opt)
        Opt = readRootDataFromFolder(Opt_folder, "EndOfEvent", 'fPrimaryZ')
        data.append(Opt)

    return data, dataDef


In [10]:
branch='fLOapprox' # The default configuration does not have stored fLSt!!! too old for that
data, dataDef =get_data_lists(branch)

Error in <TFile::Init>: file ../NSGAIIData/2307/Solutions/Sol_LStAvg/LSt_G46_AvgOpt_Vol513_230805_1core_30.root has an incorrect header length (264) or incorrect end of file length (246)
Error in <TFile::Init>: file ../NSGAIIData/2307/Solutions/Sol_LStAvg/LSt_G46_AvgOpt_Vol513_230805_1core_30.root has an incorrect header length (264) or incorrect end of file length (246)


In [11]:
# create a list for each location in Z having the impact location of each event to have a distribution for each Z segment for the box plot
import numpy as np

def segment_data(data_list, segment_values, num_segments):
    min_value = min(segment_values)
    max_value = max(segment_values)

    segment_ranges = np.linspace(min_value, max_value, num_segments + 1)

    segmented_data = [[] for _ in range(num_segments)]
    segment_values_list = [[] for _ in range(num_segments)]
    segment_averages = []
    segment_limits = []

    for i in range(len(segment_values)):
        value = segment_values[i]
        segment_index = np.searchsorted(segment_ranges, value) - 1

        if segment_index >= 0 and segment_index < num_segments:
            segmented_data[segment_index].append(data_list[i])
            segment_values_list[segment_index].append(value)

    for segment_values in segment_values_list:
        if segment_values:
            segment_averages.append(np.mean(segment_values) * 1000)
        else:
            segment_averages.append(None)

    for i in range(num_segments):
        segment_limits.append((segment_ranges[i], segment_ranges[i + 1]))

    return segmented_data, segment_values_list, segment_averages, segment_limits
def segment_data_using_limits(data_list, segment_values, segment_limits):
    num_segments = len(segment_limits)

    segmented_data = [[] for _ in range(num_segments)]
    segment_values_list = [[] for _ in range(num_segments)]
    segment_averages = []

    for i in range(len(segment_values)):
        value = segment_values[i]

        for segment_index, (lower_limit, upper_limit) in enumerate(segment_limits):
            if lower_limit <= value < upper_limit:
                segmented_data[segment_index].append(data_list[i])
                segment_values_list[segment_index].append(value)
                break

    for segment_values in segment_values_list:
        if segment_values:
            segment_averages.append(np.mean(segment_values) * 1000)
        else:
            segment_averages.append(None)

    return segmented_data, segment_values_list, segment_averages

In [12]:
boxdata, positions_list,posv,segment_limits=segment_data(data[0],data[1],12)
print(posv)
print(positions_list[0])
print(boxdata[0])
print(segment_limits)

[1.2786449999999998, 3.3244769999999995, 5.370308999999999, 7.416141, 9.461973000000002, 11.507805, 13.553636999999998, 15.599469, 17.645301000000003, 19.691133, 21.736964999999998, 23.899037454545446]
[0.000511458, 0.001022916, 0.0015343739999999998, 0.002045832, 0.000511458, 0.001022916, 0.0015343739999999998, 0.002045832, 0.000511458, 0.001022916, 0.0015343739999999998, 0.002045832, 0.000511458, 0.001022916, 0.0015343739999999998, 0.002045832, 0.000511458, 0.001022916, 0.0015343739999999998, 0.002045832]
[2760.8707856470974, 2743.0414549340526, 2736.3667435647612, 2797.5750369316265, 2767.64335964873, 2748.125305801039, 2735.9327570351725, 2782.3365008390433, 2720.7469216781524, 2727.1280857127513, 2762.2144155658725, 2778.9906861864006, 2684.880387225818, 2720.30620199051, 2759.485093620525, 2742.012979969052, 2737.3944761133253, 2703.458117538462, 2763.2496475897856, 2768.6591098042663]
[(0.0, 0.0020884535), (0.0020884535, 0.004176907), (0.004176907, 0.006265360499999999), (0.0062

In [13]:
boxdatadef, positions_listdef,posvdef=segment_data_using_limits(dataDef[0],dataDef[1],segment_limits)


In [14]:
# Example usage:

geometry_textwidth = 384  # pt
pt_to_inch = 1.0 / 72.27  # Conversion factor from points to inches
plot_width = geometry_textwidth * pt_to_inch *0.5
fig, ax = plt.subplots(figsize=(plot_width, plot_width*0.9))

# Create the boxplot
bp = ax.boxplot(boxdata,positions=posv,widths=(posv[1]-posv[0])*0.85, showfliers=True,
                         patch_artist=False,  # Set patch_artist to False
                         whiskerprops=dict(color='blue', linewidth=0.5), capprops=dict(color='blue', linewidth=0.5),
                         medianprops=dict(color='blue', linewidth=0.5), flierprops=dict(marker='o', markersize=1,
                                                                         markerfacecolor='black'), whis=1.5,boxprops=dict(color='blue', linewidth=0.5))
# Create the boxplot
bp = ax.boxplot(boxdatadef,positions=posv,widths=(posv[1]-posv[0])*0.85, showfliers=False,
                         patch_artist=False,  # Set patch_artist to False
                         whiskerprops=dict(color='red', linewidth=0.5), capprops=dict(color='red', linewidth=0.5),
                         medianprops=dict(color='red', linewidth=0.5), flierprops=dict(marker='o', markersize=1,
                                                                         markerfacecolor='black'), whis=1.5,boxprops=dict(color='blue', linewidth=0.5))
# Set x-axis tick positions and labels
ax.set_xticks(posv)
ax.set_xticklabels([f'{x:.1f}' for x in posv], rotation=45, ha='right')  # Rotate by 45 degrees
#ax.set_xticklabels([f'{x:.1f}' for x in posv])
# Calculate the average of each list in boxdata
averages = [np.mean(data) for data in boxdata]

# Add red scatter points for the averages
ax.scatter(posv, averages, color='red', s=6, label='Mean')
ax.grid()

plt.savefig("Figures/2307/NSGAII/ZvsLO_AVG_LST.pdf", bbox_inches='tight')
plt.show()


/tmp/ipykernel_43149/4113380731.py:32: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [15]:
# Example usage:
branch='fLSt' # The default configuration does not have stored fLSt!!! too old for that
data, dataDef =get_data_lists(branch)
boxdata, positions_list,posv,segment_limits=segment_data(data[0],data[1],12)

geometry_textwidth = 384  # pt
pt_to_inch = 1.0 / 72.27  # Conversion factor from points to inches
plot_width = geometry_textwidth * pt_to_inch *0.5
fig, ax = plt.subplots(figsize=(plot_width, plot_width*0.9))

# Create the boxplot
bp = ax.boxplot(boxdata,positions=posv,widths=(posv[1]-posv[0])*0.85, showfliers=True,
                         patch_artist=False,  # Set patch_artist to False
                         whiskerprops=dict(color='blue', linewidth=0.5), capprops=dict(color='blue', linewidth=0.5),
                         medianprops=dict(color='blue', linewidth=0.5), flierprops=dict(marker='o', markersize=1,
                                                                         markerfacecolor='black'), whis=1.5,boxprops=dict(color='blue', linewidth=0.5))
# Create the boxplotdth=0.5))
# Set x-axis tick positions and labels
ax.set_xticks(posv)
ax.set_xticklabels([f'{x:.1f}' for x in posv], rotation=45, ha='right')  # Rotate by 45 degrees
#ax.set_xticklabels([f'{x:.1f}' for x in posv])
# Calculate the average of each list in boxdata
averages = [np.mean(data) for data in boxdata]

# Add red scatter points for the averages
ax.scatter(posv, averages, color='red', s=6, label='Mean')
ax.grid()

plt.savefig("Figures/2307/NSGAII/ZvsLST_AVG_LST.pdf", bbox_inches='tight')
plt.show()


/tmp/ipykernel_43149/4159775105.py:30: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
Error in <TFile::Init>: file ../NSGAIIData/2307/Solutions/Sol_LStAvg/LSt_G46_AvgOpt_Vol513_230805_1core_30.root has an incorrect header length (264) or incorrect end of file length (246)
Error in <TFile::Init>: file ../NSGAIIData/2307/Solutions/Sol_LStAvg/LSt_G46_AvgOpt_Vol513_230805_1core_30.root has an incorrect header length (264) or incorrect end of file length (246)


In [16]:
# Create a new list of lists to store the multiplied data
multiplied_boxdatadef = []

# Multiply every element in every list in boxdatadef by 3 and add to the new list
for sublist in boxdatadef:
    multiplied_sublist = [x * 3 for x in sublist]
    multiplied_boxdatadef.append(multiplied_sublist)

In [17]:
# Example usage:a
branch='fLSt' # The default configuration does not have stored fLSt!!! too old for that
data, dataDef =get_data_lists(branch)
boxdata, positions_list,posv,segment_limits=segment_data(data[0],data[1],12)

geometry_textwidth = 384  # pt
pt_to_inch = 1.0 / 72.27  # Conversion factor from points to inches
plot_width = geometry_textwidth * pt_to_inch *0.5
fig, ax = plt.subplots(figsize=(plot_width, plot_width*0.9))

# Create the boxplot
bp = ax.boxplot(boxdata,positions=posv,widths=(posv[1]-posv[0])*0.85, showfliers=True,
                         patch_artist=False,  # Set patch_artist to False
                         whiskerprops=dict(color='blue', linewidth=0.5), capprops=dict(color='blue', linewidth=0.5),
                         medianprops=dict(color='blue', linewidth=0.5), flierprops=dict(marker='o', markersize=1,
                                                                         markerfacecolor='black'), whis=1.5,boxprops=dict(color='blue', linewidth=0.5))

# Create the boxplot
bp = ax.boxplot(multiplied_boxdatadef,positions=posv,widths=(posv[1]-posv[0])*0.85, showfliers=False,
                         patch_artist=False,  # Set patch_artist to False
                         whiskerprops=dict(color='red', linewidth=0.5), capprops=dict(color='red', linewidth=0.5),
                         medianprops=dict(color='red', linewidth=0.5), flierprops=dict(marker='o', markersize=1,
                                                                         markerfacecolor='black'), whis=1.5,boxprops=dict(color='blue', linewidth=0.5))
# Create the boxplotdth=0.5))
# Set x-axis tick positions and labels
ax.set_xticks(posv)
ax.set_xticklabels([f'{x:.1f}' for x in posv], rotation=45, ha='right')  # Rotate by 45 degrees
#ax.set_xticklabels([f'{x:.1f}' for x in posv])
# Calculate the average of each list in boxdata
averages = [np.mean(data) for data in boxdata]

# Add red scatter points for the averages
ax.scatter(posv, averages, color='red', s=6, label='Mean')
ax.grid()
ax.set_xlabel('Length (\si{\milli\meter})')
ax.set_ylabel(r"$L_{SP}$ $\left(\si{\gamma_d\milli\meter/\mega\electronvolt}\right)$")  # Set x-axis label with units

plt.savefig("Figures/2307/NSGAII/ZvsLST_AVG_LST_Both.pdf", bbox_inches='tight')
plt.show()


/tmp/ipykernel_43149/4039989507.py:39: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
  plt.show()
Error in <TFile::Init>: file ../NSGAIIData/2307/Solutions/Sol_LStAvg/LSt_G46_AvgOpt_Vol513_230805_1core_30.root has an incorrect header length (264) or incorrect end of file length (246)
Error in <TFile::Init>: file ../NSGAIIData/2307/Solutions/Sol_LStAvg/LSt_G46_AvgOpt_Vol513_230805_1core_30.root has an incorrect header length (264) or incorrect end of file length (246)
